In [1]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense,BatchNormalization
from keras.models import Sequential,load_model
from keras.optimizers import adam
from keras.preprocessing import image
import cv2
import datetime
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [13]:
# creating sequential model for our data
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer = 'he_uniform',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='relu',kernel_initializer = 'he_uniform'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer = 'he_uniform'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer = 'he_uniform'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
# compile our model with appropriate hyper paarameters
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# Creating generator for our training and testing dataset
train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
        'data/train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')
test_set = test_datagen.flow_from_directory(
        'data/test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')
# fitting the dataset to our model with 10 epochs
model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,
        )

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.
Epoch 1/10
83/83 [==============================] - 41s 495ms/step - loss: 0.7070 - accuracy: 0.8699 - val_loss: 0.4175 - val_accuracy: 0.9588
Epoch 2/10
83/83 [==============================] - 39s 471ms/step - loss: 0.0947 - accuracy: 0.9642 - val_loss: 0.0580 - val_accuracy: 0.9794
Epoch 3/10
83/83 [==============================] - 38s 463ms/step - loss: 0.0602 - accuracy: 0.9787 - val_loss: 0.0658 - val_accuracy: 0.9639
Epoch 4/10
83/83 [==============================] - 38s 461ms/step - loss: 0.0567 - accuracy: 0.9833 - val_loss: 0.0189 - val_accuracy: 0.9639
Epoch 5/10
83/83 [==============================] - 39s 464ms/step - loss: 0.0482 - accuracy: 0.9848 - val_loss: 0.5326 - val_accuracy: 0.9330
Epoch 6/10
83/83 [==============================] - 39s 464ms/step - loss: 0.0320 - accuracy: 0.9901 - val_loss: 6.9000e-04 - val_accuracy: 0.9330
Epoch 7/10
83/83 [==============================] - 39s

In [16]:
#To test for individual images: 0 means person is wearing mask in the image/ 1 means person is not wearing mask.
test_image=image.load_img(r'C:\Users\tanya\Desktop\ml2\Project\data\test\with_mask\5-with-mask.jpg',
                          target_size=(150,150,3))
test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
model.predict_classes(test_image)[0][0]

0

In [18]:
# Webcam module
video_capture=cv2.VideoCapture(0)
# detection of face using pre trained model
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Webcam will start capturing all frames
while video_capture.isOpened():
    _,img=video_capture.read()
    # detection of face with defined hyper parameters
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        #Set webcam window name as temp.jpg
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        # predicting whether face is covered with mask or not
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=model.predict_classes(test_image)[0][0]
        # if predition is 1 it will show face with red rectangle stating not wearing mask
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NOT WEARING MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        # if predition is 0 it will show face with green rectangle stating wearing mask
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'WAERING MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('video',img)
    #Stop capturing frames when pressing 'q' key
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break
    
video_capture.release()
cv2.destroyAllWindows()